In [2]:
import pandas as pd

## Group age brackets


## Convert Age brackets to ratios


In [16]:
import pandas as pd

def group_age_brackets(df, age_brackets: list[tuple]) -> pd.DataFrame:
    """Creates new columns with the age brackets grouped by the given age brackets.
    If there are gaps between the brackets, raises an error listing those gaps."""
    new_df = df.copy()
    
    # Sort the age brackets by the lower age
    age_brackets.sort()

    # Check for gaps between the age brackets
    gaps = []
    prev_upper_age = -1  # Start with -1 to include 0
    for lower_age, upper_age in age_brackets:
        if prev_upper_age < lower_age - 1:
            gaps.append(f"{prev_upper_age + 1} -> {lower_age - 1}")
        prev_upper_age = upper_age

    if gaps:
        raise ValueError(f"Error -> there is a gap between age ranges: {', '.join(gaps)} are not enclosed in given age_brackets")

    for lower_age, upper_age in age_brackets:
        # Initialize a new column for the age bracket with zeros
        new_df[f'{lower_age}-{upper_age}'] = 0
        for column in df.columns:
            if column.startswith('P'):
                # Extract age from column name
                age_part = column.split('_')[1]
                if age_part.isdigit():
                    age = int(age_part)
                elif age_part == 'and_ov':  # Handle 'and_over' case
                    age = upper_age + 1  # Include this column in all brackets that include and exceed this age
                else:
                    # If the column name does not follow expected format, skip it
                    continue
                
                # Check if the age falls within the current age bracket
                if lower_age <= age <= upper_age:
                    new_df[f'{lower_age}-{upper_age}'] += df[column]

    return new_df

# Example use
example_df = pd.DataFrame({'P50_54': [318, 569], 'P55_59': [355, 594], 'P85_and_ov': [87, 113]})
example_age_brackets = [(0, 58)]
result_df = group_age_brackets(example_df, example_age_brackets)
print(result_df)

   P50_54  P55_59  P85_and_ov  0-58
0     318     355          87   318
1     569     594         113   569
